# Formalne metode u softverskom inženjeringu
## Neuronske mreže i LightGBM, projekat br. 2
### Saša Savić 1153/20

In [4]:
### Neuronska mreža

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_rcv1
from sklearn.metrics import precision_score, recall_score

# učitavanje podataka (1000 podataka, iz nekog rezloga crash-uje kernel ako stavim više), standardizacija...

rcvdata = fetch_rcv1()
X = rcvdata.data[0:1000].toarray()
Y = rcvdata.target[0:1000].toarray()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.5, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definicija modela, slojevi n. mreža, funkcija greške
class Network(nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.layer1 = nn.Linear(input_size, 512)
        self.layer2 = nn.Linear(512, 1024)
        self.dropout = nn.Dropout(0.2)
        self.layer3 = nn.Linear(1024, 512)
        self.layer4 = nn.Linear(512, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # dodajemo sigmoid funkciju ovdje!

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.dropout(x)  
        x = self.relu(self.layer3(x))
        x = self.sigmoid(self.layer4(x))  # primjena sigmoida na izlaz
        return x

# kreiranje modela
input_size = X_train_tensor.shape[1]
output_size = Y_train_tensor.shape[1]
model = Network(input_size, output_size)

criterion = nn.BCELoss()  # obzirom da koristimo sigmoid u forward metodi, koristimo BCELoss
optimizer = optim.Adam(model.parameters()) # koristimo Adam za optimizaciju

# obucavanje modela
def train_model(model, train_loader, test_loader, epochs=10):
    best_precision = 0
    best_recall = 0
    best_model_state = None
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for X_batch, Y_batch in train_loader:
            optimizer.zero_grad()  # moramo resetovati gradijent

            outputs = model(X_batch)
            loss = criterion(outputs, Y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * X_batch.size(0)

        # Evaluacija 
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for X_batch, Y_batch in test_loader:
                outputs = model(X_batch)
                preds = (outputs > 0.5).float()
                all_preds.append(preds)
                all_labels.append(Y_batch)
        
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        precision = precision_score(all_labels.cpu().numpy(), all_preds.cpu().numpy(), average='micro', zero_division=0)
        recall = recall_score(all_labels.cpu().numpy(), all_preds.cpu().numpy(), average='micro', zero_division=0)

        if precision > best_precision:
            best_precision = precision
            best_recall = recall
            best_model_state = model.state_dict()

        print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss/len(train_loader.dataset):.4f} - Precision: {precision:.4f}, Recall: {recall:.4f}')
    
    torch.save(best_model_state, 'best_model.pth')
    return best_precision, best_recall

# Obučavanje modela i ispis najboljeg modela
best_precision, best_recall = train_model(model, train_loader, test_loader, epochs=10)

print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')



Epoch 1/10 - Loss: 0.4139 - Precision: 0.3168, Recall: 0.2139
Epoch 2/10 - Loss: 0.2761 - Precision: 0.6338, Recall: 0.1798
Epoch 3/10 - Loss: 0.1206 - Precision: 0.8413, Recall: 0.2448
Epoch 4/10 - Loss: 0.0804 - Precision: 0.7997, Recall: 0.3420
Epoch 5/10 - Loss: 0.0592 - Precision: 0.8244, Recall: 0.3906
Epoch 6/10 - Loss: 0.0427 - Precision: 0.8261, Recall: 0.4417
Epoch 7/10 - Loss: 0.0307 - Precision: 0.7750, Recall: 0.4666
Epoch 8/10 - Loss: 0.0215 - Precision: 0.8120, Recall: 0.4933
Epoch 9/10 - Loss: 0.0136 - Precision: 0.8000, Recall: 0.5103
Epoch 10/10 - Loss: 0.0099 - Precision: 0.7676, Recall: 0.4976
Best Precision: 0.8413
Best Recall: 0.2448


In [5]:
### LightGBM

In [3]:
import lightgbm as lgb
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Izbacili smo ponovo korištenje test-train splita, dobijeni bolji rezultati metrika evaluacije na kraju

# Pretvaranje Y_train i Y_test u numeričke oznake klasa, kompatabilno sa LightGBM-om
Y_train_np = np.argmax(Y_train, axis=1)
Y_test_np = np.argmax(Y_test, axis=1)

# Definisanje za pretragu hiperparametara
def objective(trial):
    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'num_class': len(np.unique(Y_train_np)),  # Broj klasa!
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'verbosity': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, Y_train_np)
    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test_np, Y_pred)
    return accuracy

# Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

best_params = trial.params
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, Y_train_np)

# Evaluacija
Y_pred = best_model.predict(X_test)
precision = precision_score(Y_test_np, Y_pred, average='macro')
recall = recall_score(Y_test_np, Y_pred, average='macro')
accuracy = accuracy_score(Y_test_np, Y_pred)

print(f'Best model - Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}')


[I 2024-08-26 22:09:42,217] A new study created in memory with name: no-name-bd8e631c-bd34-42ec-b2b0-be56126b1eb7
[I 2024-08-26 22:09:44,146] Trial 0 finished with value: 0.392 and parameters: {'learning_rate': 0.09087492225724444, 'num_leaves': 185, 'max_depth': 12, 'feature_fraction': 0.6584634539263836, 'bagging_fraction': 0.6926363359711982, 'bagging_freq': 6, 'lambda_l1': 9.470097529414828, 'lambda_l2': 3.2776989773176757}. Best is trial 0 with value: 0.392.
[I 2024-08-26 22:09:46,224] Trial 1 finished with value: 0.594 and parameters: {'learning_rate': 0.06806684428651762, 'num_leaves': 87, 'max_depth': 8, 'feature_fraction': 0.7544514463951049, 'bagging_fraction': 0.6483956931185013, 'bagging_freq': 7, 'lambda_l1': 0.5667798653401083, 'lambda_l2': 7.674348548898353}. Best is trial 1 with value: 0.594.
[I 2024-08-26 22:09:48,176] Trial 2 finished with value: 0.61 and parameters: {'learning_rate': 0.07568586121376299, 'num_leaves': 183, 'max_depth': 8, 'feature_fraction': 0.912088

Best trial:
  Value: 0.61
  Params: 
    learning_rate: 0.07568586121376299
    num_leaves: 183
    max_depth: 8
    feature_fraction: 0.9120880376799133
    bagging_fraction: 0.4768025137209355
    bagging_freq: 7
    lambda_l1: 1.0429400679855008
    lambda_l2: 2.0841114886155925
Best model - Precision: 0.3174, Recall: 0.2958, Accuracy: 0.6100


/home/vikingg89/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
###